In [1]:
%load_ext autoreload 
%autoreload 2

In [4]:
import pandas as pd
import numpy as np

from src.data_processing.processors.Preprocessor import Preprocessor


In [134]:
lat_path = "/projects/p31961/gaby_all_raw_data/AA_Latencies.xlsx"
sex_path = "/projects/p31961/gaby_all_raw_data/AA_ListofSex.xlsx"


lat_data = pd.read_excel(lat_path)
sex_data = pd.read_excel(sex_path)
data = pd.read_parquet('/projects/p31961/gaby_data/aggregated_data/downsampled_aggregated_data.parquet.gzp')

mouse_id    category
day            int64
event         object
sensor        object
time         float32
trial          int32
signal       float32
dtype: object

In [136]:
def tweak_sex_data(df):

    return (
        df
        .rename(columns = lambda c: c.replace(' ', '_').lower())
        .assign(mouse_id = lambda df: df['mouse_id'].str.replace("-", "_").astype('category')
        )
    )
sex_df = tweak_sex_data(sex_data)

In [135]:
lat_data

,Mouse,Day,Trial,Latency,Event
0,309-910,1,1,5.69,Escape
1,309-910,1,2,6.07,Escape
2,309-910,1,3,9.65,Escape
3,309-910,1,4,12.62,Escape
4,309-910,1,5,6.70,Escape
...,...,...,...,...,...
2935,152-071,7,26,2.78,Avoid
2936,152-071,7,27,4.80,Avoid
2937,152-071,7,28,1.89,Avoid
2938,152-071,7,29,3.64,Avoid


In [128]:
def tweak_lat_data(df):
    return (df.
            rename(columns = lambda c: c.replace(' ', '_').lower())
            .rename(columns = {'mouse_': 'mouse_id'})
            .assign(mouse_id = lambda df_: df_['mouse_id'].str.replace("-", "_").astype('category'), 
                    event = lambda df_: df_['event'].str.lower())
    )
lat_df = tweak_lat_data(lat_data)

,mouse_id,day,trial,latency,event
0,309_910,1,1,5.69,escape
1,309_910,1,2,6.07,escape
2,309_910,1,3,9.65,escape
3,309_910,1,4,12.62,escape
4,309_910,1,5,6.70,escape
...,...,...,...,...,...
2935,152_071,7,26,2.78,avoid
2936,152_071,7,27,4.80,avoid
2937,152_071,7,28,1.89,avoid
2938,152_071,7,29,3.64,avoid


In [149]:
all_data = (data
            .dropna()
            .merge(lat_df, on = ['mouse_id', 'day', 'trial', 'event'], how = 'left')
            .assign(latency = lambda df: df['latency'].fillna(0))
            .merge(sex_df, on = 'mouse_id', how = 'left')
)
all_data



,mouse_id,day,event,sensor,time,trial,signal,latency,sex
0,312_257,5,cue,D2,-25.000000,0,-0.155359,0.00,F
1,312_257,5,cue,D2,-24.901531,0,-0.420553,0.00,F
2,312_257,5,cue,D2,-24.803064,0,-1.592294,0.00,F
3,312_257,5,cue,D2,-24.704596,0,-1.268734,0.00,F
4,312_257,5,cue,D2,-24.606127,0,-0.210176,0.00,F
...,...,...,...,...,...,...,...,...,...
5858795,142_238,4,escape,D1,19.606127,1,0.299603,7.54,F
5858796,142_238,4,escape,D1,19.704596,1,-0.249408,7.54,F
5858797,142_238,4,escape,D1,19.803064,1,-0.486369,7.54,F
5858798,142_238,4,escape,D1,19.901531,1,-0.146374,7.54,F
